<a href="https://colab.research.google.com/github/7b809/logic-gems/blob/main/flask_app_loadded_tunneled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask
!curl -fsSL https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared


In [3]:
# ============================================
# 🚀 Flask + Cloudflare Tunnel (One-Cell Version)
# ============================================

from flask import Flask
import subprocess, re, time, threading

# ------------------------------
# 1️⃣ Define Flask app
# ------------------------------
app = Flask(__name__)

@app.route('/')
def home():
    return "Server running successfully!"


# ------------------------------
# 2️⃣ Run Flask server in a thread
# ------------------------------
def run_flask():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # ensures it stops with the main process
flask_thread.start()

time.sleep(3)  # small wait to ensure Flask starts

# ------------------------------
# 3️⃣ Start Cloudflare Tunnel
# ------------------------------
print("⏳ Starting Cloudflare Tunnel... Please wait 10–20 seconds.\n")

process = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:5000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

public_url = None

# ------------------------------
# 4️⃣ Capture the public URL
# ------------------------------
for line in iter(process.stdout.readline, ""):
    print(line.strip())
    if "trycloudflare.com" in line:
        match = re.search(r"https://[0-9a-zA-Z\-]+\.trycloudflare\.com", line)
        if match:
            public_url = match.group(0)
            print(f"\n✅ Public URL (Access your Flask app here): {public_url}\n")
            break

# ------------------------------
# 5️⃣ Keep tunnel alive
# ------------------------------
print("🌐 Tunnel is active. Press STOP (in Colab) to terminate it.")
process.wait()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


⏳ Starting Cloudflare Tunnel... Please wait 10–20 seconds.

2025-10-22T12:54:32Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-22T12:54:32Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-22T12:54:35Z INF +--------------------------------------------------------------------------------------------+
2025-10-22T12:54:35Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-22T12:54:35Z 

INFO:werkzeug:127.0.0.1 - - [22/Oct/2025 12:54:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2025 12:54:43] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2025 13:01:54] "GET / HTTP/1.1" 200 -


KeyboardInterrupt: 